# Part 1: Vectorized Computation using Numpy

We have just learned about the logical set-up of a computer and which factors affect the performance of it. Keeping this in mind, we will learn how we can improve the performance of our code in Python.

### 0. Numpy Warm Up

In [ ]:
import numpy as np

__E 2.3:__ Create a null vector of size 10 called `zeros`.

**E:** Assign the value `2` to the fifth element of the array `zeros`. 

__E 2.5:__ Assign the value 1 to every second element the array `zeros`. 

**FYI:** To get some more practice in using numpy, do some more exercises which suite your skill level from [Nicolas Rougier's 100 Numpy Exercises](https://github.com/rougier/numpy-100).

## 1. For-Loop vs Vectorized Computation

### The theme of the EuroSciPy Conference 2018 was: _"Vectorized is better than an explict loop"_ 

<img src="img/IMG_20190807_082235.jpg" alt="tshirt" width="600"/>

### Example: Calculate the sum of two list / arrays

**E:** Create two lists which contain the numbers from 0 to 1000. 

In [ ]:
listx = list(range(1001))
list1 = listx
list2 = listx

**E:** Caculate the sum of the two lists and measure the execution time using the magic command `%%timeit`

In [ ]:
sums_list = []
i = 0
for i in listx:
    sums_list.append(list1[i] + list2[i])

In [ ]:
sums_list

In [ ]:
%%timeit
sums_list = []
i = 0
for i in listx:
    sums_list.append(list1[i] + list2[i])
    i += 1

**E:** Convert the lists to numpy arrays and perform the calculation again using the same method as above. Measure the execution time.

In [ ]:
import numpy as np

arr1 = np.array(list1)
arr2 = np.array(list2)

In [ ]:
# sums_arr = []
# i = 0
# for i in list:
#     sums_arr.append(arr1[i] + arr2[i])

In [ ]:
# %%timeit
# sums_arr = []
# i = 0
# for i in list:
#     sums_arr.append(arr1[i] + arr2[i])
#     i += 1

**E:** Perform the calculation using a numpy ufunc and measure the execution time. Compare the exectuion times of all methods.

In [ ]:
%%timeit
np.add(arr1, arr2)

### Can the logical setup of a computer explain the differences?

<img src="./img/vectorization.png" alt="setup" width="600"/>

### Lists vs Numpy Arrays

Arrays are better than lists for continous reading of data, because lists usually only store pointers to variables stored in memory, not the variable values themselves. This can lead to bad spatial locality when reading the data, because the data might be spread all over the memory.

<img src="https://i.stack.imgur.com/K26b0.png" alt="drawing" width="600"/>


#### What about `np.vectorize`?

There is a `numpy.vectorize()` functions which seems like it vectorizes your Python code. Execute the code below and compare the results to the previous ones.

In [ ]:
def add_values(a, b):
    "Adds a and b"
    return a + b

In [ ]:
add_values_vect = np.vectorize(add_values)

In [ ]:
%%timeit
add_values_vect(values1, values2)

**E:** Take a look at [documentation of the numpy.vectorize() fucntion](https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html). How does `numpy.vectorize()` perform the calculation?

__Bonus:__ Write the function above as a lambda function. 

### Vectorizing your script using Numpy 

* Avoid for loops (or even nested for loops) when possible. 
* Perform __calculations on arrays__ instead of scalars
* Use Numpy's built-in universal functions whenever possible to enable vectorized computation. 

**Conclusion:** Use lists if the elements are of different data type or you do not need to perform any calculations. Otherwise use numpy arrays.

## 2. Spatial Locality

**Exercise** Create a 2-dimensional array with random numbers with 10000 rows and 10000 columns. 

In [ ]:
import numpy as np

rand_arr = np.random.randint(100, size=(5000, 10000))

**By the way:** How can you make the function always yields the same random numbers for better reproducibility?

In [ ]:
np.random.seed(42)
rand_arr = np.random.randint(100, size=(5000, 10000))
rand_arr

**Exercise:** Calculate the mean of each row and the mean of each column. Measure how long each of these operations takes. What do you observe?

In [ ]:
np.mean(rand_arr[1])

In [ ]:
np.mean(rand_arr[0])

### Looking at the way arrays are stored in pyhsical memory might help us explain this phenomenon. 

![spatial_locality_example1](img/spatial_locality_example.png)
![spatial_locality_example2](img/spatial_locality_example2.png)
![spatial_locality_example3](img/spatial_locality_example3.png)

Source: [Spatial locality explained](http://www.tophatstuff.co.uk/index.html@p=119.html)


### Row and Column Major Order

By default numpy arrays are stored in C order (row-major), not in F order (column-major). Therefore, calculations along the rows (axis=1) are faster than along the columns (axis=0) for a 2D array. So calculating the mean of each row is faster than the mean of each column if the numpy array is stored as default in C order.

&rarr; [SciPy Lectures: C/F order](https://scipy-lectures.org/advanced/advanced_numpy/index.html#c-and-fortran-order)

<img src="img/row-major-order.png" alt="row-major" width="500"/>


In [ ]:
arr2D = np.arange(0, 50).reshape(10, 5)

In [ ]:
arr2D.strides
# Angaben in Byte

**E:** Convert the 2d array into a 1D array using F order.

In [ ]:
arr2D.ravel(order="F")

In [ ]:
arr2D.ravel(order="C")

## 4. Numpy Strides
"Strides are the number of bytes to jump-over in the memory in order to get from one item to the next item along each direction/dimension of the array. In other words, it's the byte-separation between consecutive items for each dimension." [Stack Overflow](https://stackoverflow.com/questions/53097952/how-to-understand-numpy-strides-for-layman)

--> [SciPy Lectures: Strides](https://scipy-lectures.org/advanced/advanced_numpy/index.html#indexing-scheme-strides)

__E 2.1:__ Create an array `arr` with values ranging from 10 to 49 of data type "int8" (8-bit integer).

**E:** Print the strides of `arr`.

In [ ]:
arr.strides

**E:** change the data type of `arr` to 16-bit integer and print the strides again. Why do they change? 

**E:** Reshape the array so that it has 4 rows and 10 columns and print the strides again.

**Explanation:** To get to the next row you need to jump 10 * 2 bytes (16 bit integer = 2 byte). To get to the next column you need to jump 1 * 2 bytes.

__E:__ What happens to the data in memory when you transpose the array `arr3`? 

In [ ]:
 arr3.transpose().strides

### Bonus exercise:
Create an F ordered array with 15 columns and 10 rows with data type 32bit float. Calculate the strides by yourself. Check your result by executing the `strides`. 

## Summary

* You should be able now to explain the sentence "vectorized is better than an explicit loop".
* When performing calculations it is good to keep in mind how arrays are stored in memory, since it can influence your processing time. 
* Most of the time however, NumPy or Python take care of it.

### Resources:

__Micha Gorelick, Ian Ozsvald. High Performance Python. O'Reilly, 2014. (Safari Books)__ (very good)

Schmidt, B., Gonzalez-Dominguez, J., Hundt, C., & Schlarb, M. (2017). Parallel programming: concepts and practice. Morgan Kaufmann. [Google Books](https://books.google.de/books?hl=en&lr=&id=-y9HDgAAQBAJ&oi=fnd&pg=PP1&dq=Parallel+Programming:+Concepts+and+Practice&ots=vPL3kPvTkJ&sig=gN5EA0okkbtUHufPvTB37MAGFis)

[Spatial locality explained](http://www.tophatstuff.co.uk/index.html@p=119.html)

[SciPy Lecture Notes: Advanced Numpy](https://scipy-lectures.org/advanced/advanced_numpy/index.html)
 
[Real Python: Numpy array programming](https://realpython.com/numpy-array-programming/)

[NumPy Tutorial]( https://github.com/mingrammer/cs231n-numpy-tutorial/blob/master/numpy_tutorial.ipynb
)

[Memory Bound Problems](https://github.com/ASPP/MemoryBoundComputations)